
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 



In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [5]:
import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

[I 2025-06-11 17:14:01,502] A new study created in memory with name: test


17:14:01 redisvl.index.index INFO   Index already exists, not overwriting.
17:14:01 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
17:14:02 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.23it/s]

17:14:02 root INFO   Data indexed total_indexing_time=0.989s, num_docs=3633


17:14:03 root INFO   Saving metrics for study: 88629332-3769-4ed6-9cec-2c950d5bbdb8, METRICS={'search_method': ['hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector'], 'total_indexing_time': [0.995, 1.126, 1.238, 1.238, 1.251, 1.479, 1.479, 1.479, 0.989, 0.989, 0.989], 'avg_query_time': [0.001379058087942401, 0.0013568142988364393, 0.0014326011433320887, 0.0012770201030530427, 0.0012413067714348667, 0.0012326343878872994, 0.0011889233308679918, 0.0012138657525596972, 0.0012014170549233263, 0.0010982010017607604, 0.0007533724463022899], 'model': ['sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-trans

...

Batches: 100%|██████████| 1/1 [00:00<00:00, 64.48it/s]

17:14:31 root INFO   Data indexed total_indexing_time=1.485s, num_docs=3633


17:14:31 root INFO   Saving metrics for study: 88629332-3769-4ed6-9cec-2c950d5bbdb8, METRICS={'search_method': ['hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'hybrid', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector'], 'total_indexing_time': [0.995, 1.126, 1.238, 1.238, 1.251, 1.479, 1.479, 1.479, 0.989, 0.989, 0.989, 0.989, 0.989, 0.989, 1.169, 1.169, 1.169, 1.169, 0.896, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485, 1.485], 'avg_query_time': [0.001379058087942401, 0.0013568142988364393, 0.0014326011433320887, 0.0012770201030530427, 0.0012413067714348667, 0.0012326343878872994, 0.0011889233308679918, 0.0012138657525596972, 0.0012014170549233263, 0.0010982010017607604, 0.0007533724463022899, 0.0007078160442434967, 0.0011058761608489895, 0.0010777922237620635, 0.0013610571

[I 2025-06-11 17:14:31,624] Trial 19 finished with value: 1.485 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float32', 'distance_metric': 'cosine', 'ret_k': 2, 'ef_runtime': 20, 'ef_construction': 300, 'm': 16}. Best is trial 9 with value: 1.485.


17:14:31 root INFO   Completed Bayesian optimization... 


17:14:31 root INFO   Best Configuration: 9: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float32', 'distance_metric': 'cosine', 'ret_k': 5, 'ef_runtime': 30, 'ef_construction': 300, 'm': 64}:


17:14:31 root INFO   Best Score: [1.485]




In [6]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "total_indexing_time", "f1@k"]].sort_values(by="f1@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,total_indexing_time,f1@k
0,hybrid,hnsw,float32,100,30,8,0.001379,0.995,0.130712
8,hybrid,hnsw,float16,150,30,8,0.001201,0.989,0.130712
17,hybrid,hnsw,float32,200,10,8,0.001224,1.169,0.130712
1,hybrid,hnsw,float16,200,50,16,0.001357,1.126,0.130712
14,hybrid,hnsw,float32,200,30,8,0.001361,1.169,0.130712
13,hybrid,hnsw,float16,100,50,16,0.001078,0.989,0.130712
12,hybrid,hnsw,float16,200,50,8,0.001106,0.989,0.130712
9,hybrid,hnsw,float16,150,30,16,0.001098,0.989,0.130712
15,hybrid,hnsw,float32,150,20,64,0.001259,1.169,0.130712
7,hybrid,hnsw,float32,300,30,16,0.001214,1.479,0.130712
